## 인천 나리타 노선 가격 정보 읽어오기
- 대상 사이트 : 대한항공 국제선 예매 사이트

### 대한항공 사이트 웹 크롤링
    - 대한항공 국제선 항공권예매 페이지를 이용한 크롤링
    - 세션을 생성하여 세션을 이용해 RESTFull json API 페이지 호출 하여 정보 획득
    - 필요 데이터 추출 하여 Pandas 의 DataFrame 형태로 생성
    - 생성된 데이터 엑셀 파일로 저장

In [8]:
from IPython.display import display
import requests
import pandas as pd
import numpy as np
from pandas import DataFrame
from bs4 import BeautifulSoup
import time
from datetime import datetime
from datetime import timedelta
from common.crawling_util import session_crawling
from common.parsing_util import stat_fare

def crawling_KE_data(dpt,arr,dpt_date):
    ##출발지, 도착지, 출발일을 기준으로 국내선(국제선) 편도 가격 읽어오기
    print('Crawling koreanair homepage schedule site')
    session_url = "https://www.koreanair.com/korea/ko/booking/booking-gate.html?intl#international"
    session_head = {
        'Accept-Language':'ko-KR',
        'Referer':'https://www.koreanair.com/korea/ko.html',
        'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    }
    
    '''
    session_url = "https://www.koreanair.com/korea/ko/booking/xiow.html"
    session_head = {
        'Referer':'https://www.koreanair.com/korea/ko/booking/booking-gate.html?intl',
        'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    }
    '''
    ## Rest Full 방식 https://www.koreanair.com/api/fly/revenue/from/ICN/to/NRT/on/06-15-2017-0000
    url = "https://www.koreanair.com/api/fly/revenue/from/{dpt}/to/{arr}/on/{mm}-{dd}-{yyyy}-0000"
    url_param = {
        'dpt':dpt,    'arr':arr,
        'yyyy':dpt_date[:4],
        'mm':dpt_date[4:6],
        'dd':dpt_date[6:]
    }
    url = url.format(**url_param)
    head = {
        'Accept-Language':'ko-KR', ## 중요, 영문사이트는 capcha 를 요구함
        'page-id':'/booking/xiow.html', ## 필수 항목
        'uidd':'83^51%8638461@384712',  ## 필수 항목
        'Referer':'https://www.koreanair.com/korea/ko/booking/xiow.html',
        'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    }
    param ={## 파라미터는 고정, _ 부분만 조회 시점 타임스탬프 적용
        'flexDays':'0',    'scheduleDriven':'false',
        'purchaseThirdPerson':'',
        'domestic':'false','isUpgradeableCabin':'false',
        'currency':'',     'bonusType':'SKYPASS',
        'countryCode':'',
        'adults':'1',    'children':'0',    'infants':'0',
        'cabinClass':'ECONOMY',
        'adultDiscounts':'',    'adultInboundDiscounts':'',
        'childDiscounts':'',    'childInboundDiscounts':'',
        'infantDiscounts':'',   'infantInboundDiscounts':'',
        '_':str(int(datetime.now().timestamp())),
    }

    return session_crawling(session_url,url,param,session_head=session_head,head=head,method='get',json=True)

## 해당 아웃바운드 스케줄의 가격에 대한 키 정보 읽어오기
def get_fare_key(fare_key,fare_class,raw_data):
    mapper = raw_data['tripFareMapper']
    fare_key_list = []
    for book_cls in fare_class:
        fare_key_list.extend(mapper[fare_key+'-'+book_cls[:-1]])
    return set(fare_key_list)
## 해당 아웃바운드 스케줄의 가격 정보 읽어 오기
def get_fares(fare_key,fare_class,raw_data):
    fare_keys = get_fare_key(fare_key,fare_class,raw_data)
    fare_list = []
    for key in fare_keys:
        fare = raw_data['fares'][key]['fares'][0]
        fare_list.append([fare['amount'],fare['fuelSurcharge'],fare['tax']])
    return sorted(fare_list)
## 국내선용 파싱 정보
data_heads = ['date','flt','dpt','arr','fare1','fare2','tax1','tax2','seat']
## raw 데이터로 부터 DataFrame 생성
def raw_to_df(raw_data):
    ## 읽어온 데이터의 json['Result']['code'] 값이 0000 이 아닌 경우 오류
    sch_list = []
    for schedule in raw_data['outbound']:
        flt = schedule['flights'][0]
        sch = [flt['departure'][:4]+flt['departure'][5:7]+flt['departure'][8:10],
               flt['flightNumber'],flt['departure'][11:16],flt['arrival'][11:16]]
        fares = get_fares(schedule['key'],schedule['remainingSeatsByBookingClass'],raw_data)
        ## 조회해온 가격정보(가격,유류세,택스) 역순으로 하여 가격만 가져옮
        sch.extend([str(int(fare[0])) for fare in fares[::-1]])
        ## economy 가격만 처리(2개씩 표출), 국제선은 옵션을 적용하여 조회하고 있음, 필요시 추가하여야함
        sch.extend(['0' for _ in range(2-len(fares))])
        ## 첫번째 항목을 이용 유류세, 텍스 추가
        if len(fares) > 0:
            sch.extend([str(int(fare)) for fare in fares[0][1:]])
        else:
            sch.extend(['0','0'])
        ## ECONOMYY 클래스에 대한 좌석 수 추가 - 노멀 클래스
        sch.append(schedule['remainingSeatsByBookingClass'].get('ECONOMYY',0))
        sch_list.append(sch)
    return pd.DataFrame(sch_list,columns=data_heads)

## 하루 기준 가격정보, 텍스정보, 최소값, 최대값, 평균 DataFrame 생성
def read_KE_1day_fare(dpt,arr,dpt_date):
    ## 데이터 읽어오기
    raw_data = crawling_KE_data(dpt,arr,dpt_date)
    df=raw_to_df(raw_data)
    df = df[df['date'] == dpt_date].reset_index()[data_heads]
    if df is None or len(df) == 0: ## 읽는 도중 에러가 나거나 익셉션 발생으로 문제가 있을 경우 처리
        print('********** No Data **********')
        return None
    ## 최소값 최대값 평균 계산
    stat = stat_fare(df,columns=['fare1','fare2'])
    ## 요약정보 추가
    df.ix[len(df)] = [dpt_date,'min','max','mean',str(int(stat[0])),str(int(stat[1])),str(int(stat[2])),'','']
    return df

## 정해진 기간의 데이터 읽어오기, 기본 30일
def read_KE_date_range_fare(dpt,arr,start=0,end=31):
    # 기본 30일간 데이터 읽어서 파일로 저장
    date_range = [ (datetime.today()+timedelta(1)*i).strftime('%Y%m%d') for i in range(start,end)]
    fare_list = []
    for d in date_range:
        try:
            fare_df = read_KE_1day_fare(dpt,arr,d)
            if fare_df is not None:
                fare_list.append(fare_df)
        except Exception as e:
            print('****** Error occured : ',e)
    result = pd.concat(fare_list,ignore_index=True)
    print('++++++++++Total : ', len(result))
    ## 파일 저장
    result.to_excel('{}/{}_{}_{}_{}_{}_{}.xls'.format('excel','KE',dpt,arr,start,end,datetime.today().strftime('%Y%m%d%H%M')))
    return result

In [9]:
## 하루치 데이터 읽어오기
dpt, arr, dpt_date = 'ICN','NRT','20170615'
read_KE_1day_fare(dpt,arr,dpt_date)

Crawling koreanair homepage schedule site
Start Session crawling
make session :  https://www.koreanair.com/korea/ko/booking/booking-gate.html?intl#international
crawling :  https://www.koreanair.com/api/fly/revenue/from/ICN/to/NRT/on/06-15-2017-0000
>> Parameters
domestic:false , isUpgradeableCabin:false , cabinClass:ECONOMY , adults:1 , childInboundDiscounts: , currency: , countryCode: , adultInboundDiscounts: , purchaseThirdPerson: , infants:0 , infantDiscounts: , childDiscounts: , bonusType:SKYPASS , _:1495075670 , adultDiscounts: , flexDays:0 , scheduleDriven:false , children:0 , infantInboundDiscounts: , 
End Session crawling


,date,flt,dpt,arr,fare1,fare2,tax1,tax2,seat
0,20170615,KE703,10:10,12:30,368500,335500,0,28000,9
1,20170615,KE001,17:40,20:00,368500,335500,0,28000,9
2,20170615,KE705,18:35,20:55,368500,335500,0,28000,9
3,20170615,KE5221,07:25,09:50,286000,0,0,28000,9
4,20170615,KE5703,11:10,13:45,368500,335500,0,28000,9
5,20170615,KE5223,14:25,16:55,286000,0,0,28000,9
6,20170615,min,max,mean,286000,368500,330000,,


In [10]:
## 정해진 기간의 데이터 읽어오기
dpt, arr = 'ICN','NRT'
start,end =0, 31 ## 읽어올 데이터 기간
read_KE_date_range_fare(dpt,arr,start,end)

Crawling koreanair homepage schedule site
Start Session crawling
make session :  https://www.koreanair.com/korea/ko/booking/booking-gate.html?intl#international
crawling :  https://www.koreanair.com/api/fly/revenue/from/ICN/to/NRT/on/05-18-2017-0000
>> Parameters
domestic:false , isUpgradeableCabin:false , cabinClass:ECONOMY , adults:1 , childInboundDiscounts: , currency: , countryCode: , adultInboundDiscounts: , purchaseThirdPerson: , infants:0 , infantDiscounts: , childDiscounts: , bonusType:SKYPASS , _:1495075688 , adultDiscounts: , flexDays:0 , scheduleDriven:false , children:0 , infantInboundDiscounts: , 
End Session crawling
Crawling koreanair homepage schedule site
Start Session crawling
make session :  https://www.koreanair.com/korea/ko/booking/booking-gate.html?intl#international
crawling :  https://www.koreanair.com/api/fly/revenue/from/ICN/to/NRT/on/05-19-2017-0000
>> Parameters
domestic:false , isUpgradeableCabin:false , cabinClass:ECONOMY , adults:1 , childInboundDiscounts

,date,flt,dpt,arr,fare1,fare2,tax1,tax2,seat
0,20170518,KE001,17:40,20:00,368500,335500,0,28000,9
1,20170518,KE705,18:35,20:55,368500,335500,0,28000,9
2,20170518,min,max,mean,335500,368500,352000,,
3,20170519,KE703,10:10,12:30,368500,335500,0,28000,9
4,20170519,KE001,17:40,20:00,368500,335500,0,28000,9
5,20170519,KE705,18:35,20:55,368500,335500,0,28000,9
6,20170519,KE5221,07:25,09:50,286000,0,0,28000,9
7,20170519,KE5703,11:10,13:45,368500,335500,0,28000,9
8,20170519,KE5223,14:35,16:55,286000,0,0,28000,9
9,20170519,min,max,mean,286000,368500,330000,,


In [11]:
## 정해진 기간의 데이터 읽어오기
dpt, arr = 'ICN','NRT'
start,end =31, 45 ## 읽어올 데이터 기간
read_KE_date_range_fare(dpt,arr,start,end)

Crawling koreanair homepage schedule site
Start Session crawling
make session :  https://www.koreanair.com/korea/ko/booking/booking-gate.html?intl#international
crawling :  https://www.koreanair.com/api/fly/revenue/from/ICN/to/NRT/on/06-18-2017-0000
>> Parameters
domestic:false , isUpgradeableCabin:false , cabinClass:ECONOMY , adults:1 , childInboundDiscounts: , currency: , countryCode: , adultInboundDiscounts: , purchaseThirdPerson: , infants:0 , infantDiscounts: , childDiscounts: , bonusType:SKYPASS , _:1495080812 , adultDiscounts: , flexDays:0 , scheduleDriven:false , children:0 , infantInboundDiscounts: , 
End Session crawling
Crawling koreanair homepage schedule site
Start Session crawling
make session :  https://www.koreanair.com/korea/ko/booking/booking-gate.html?intl#international
crawling :  https://www.koreanair.com/api/fly/revenue/from/ICN/to/NRT/on/06-19-2017-0000
>> Parameters
domestic:false , isUpgradeableCabin:false , cabinClass:ECONOMY , adults:1 , childInboundDiscounts

,date,flt,dpt,arr,fare1,fare2,tax1,tax2,seat
0,20170618,KE703,10:10,12:30,368500,335500,0,28000,9
1,20170618,KE001,17:40,20:00,368500,335500,0,28000,9
2,20170618,KE705,18:35,20:55,368500,335500,0,28000,9
3,20170618,KE5221,07:25,09:50,286000,0,0,28000,9
4,20170618,KE5703,11:10,13:45,368500,335500,0,28000,9
5,20170618,KE5223,14:25,16:55,0,0,0,0,0
6,20170618,min,max,mean,286000,368500,330000,,
7,20170619,KE703,10:10,12:30,368500,335500,0,28000,9
8,20170619,KE001,17:40,20:00,368500,335500,0,28000,9
9,20170619,KE705,18:35,20:55,368500,335500,0,28000,9


In [12]:
dpt, arr = 'ICN','NRT'
start,end =45, 90 ## 읽어올 데이터 기간
read_KE_date_range_fare(dpt,arr,start,end)

Crawling koreanair homepage schedule site
Start Session crawling
make session :  https://www.koreanair.com/korea/ko/booking/booking-gate.html?intl#international
crawling :  https://www.koreanair.com/api/fly/revenue/from/ICN/to/NRT/on/07-02-2017-0000
>> Parameters
domestic:false , isUpgradeableCabin:false , cabinClass:ECONOMY , adults:1 , childInboundDiscounts: , currency: , countryCode: , adultInboundDiscounts: , purchaseThirdPerson: , infants:0 , infantDiscounts: , childDiscounts: , bonusType:SKYPASS , _:1495081458 , adultDiscounts: , flexDays:0 , scheduleDriven:false , children:0 , infantInboundDiscounts: , 
End Session crawling
Crawling koreanair homepage schedule site
Start Session crawling
make session :  https://www.koreanair.com/korea/ko/booking/booking-gate.html?intl#international
crawling :  https://www.koreanair.com/api/fly/revenue/from/ICN/to/NRT/on/07-03-2017-0000
>> Parameters
domestic:false , isUpgradeableCabin:false , cabinClass:ECONOMY , adults:1 , childInboundDiscounts

,date,flt,dpt,arr,fare1,fare2,tax1,tax2,seat
0,20170702,KE703,10:10,12:30,368500,352000,0,28000,9
1,20170702,KE001,17:40,20:00,368500,352000,0,28000,9
2,20170702,KE705,18:35,20:55,368500,352000,0,28000,9
3,20170702,KE5221,07:25,09:50,286000,0,0,28000,9
4,20170702,KE5703,11:10,13:45,368500,352000,0,28000,9
5,20170702,KE5223,14:25,16:55,286000,0,0,28000,9
6,20170702,min,max,mean,286000,368500,335500,,
7,20170703,KE703,10:10,12:30,368500,352000,0,28000,9
8,20170703,KE001,17:40,20:00,368500,352000,0,28000,9
9,20170703,KE705,18:35,20:55,368500,352000,0,28000,9
